#### This python file creates the new dataset which is used for creating streamlit dashboard.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
# Load the GeoJSON file into a GeoDataFrame
neighborhoods = gpd.read_file('Data/Boundaries - Neighborhoods.geojson')
# Load the CSV file into a DataFrame
crimes = pd.read_csv("Data/Crimes_-_2001_to_Present_4.csv", delimiter= ';')
# Convert the DataFrame to a GeoDataFrame
def new_data():
    crimes_geo = gpd.GeoDataFrame(
    crimes,
    geometry=gpd.points_from_xy(crimes.Longitude, crimes.Latitude),
    crs='EPSG:4326')
# Perform the spatial join
    joined = gpd.sjoin(crimes_geo, neighborhoods, how='left', op='within')

# Extract the desired neighborhood information
    crime_neighborhoods = joined[['ID', 'pri_neigh', 'sec_neigh']]

# Merge the neighborhood information with the original CSV file
    crimes_with_neighborhoods = crimes.merge(crime_neighborhoods, on='ID', how='left')
# Save the merged CSV file
    crimes_with_neighborhoods.to_csv('Data/Crimes_-_2001_to_Present_with_neighborhoods.csv', index=False)

new_data()

/Users/akshatha/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3460: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
df = pd.read_csv("Data/Crimes_-_2001_to_Present_with_neighborhoods.csv", delimiter= ',')    
df['Date'] = pd.to_datetime(df['Date'])
df['time'] = [d.time() for d in df['Date']]
df['day'] = df['Date'].dt.day_name()
df['month'] = df['Date'].dt.month
median_longitude = df["Longitude"].median()
df["Longitude"] = df["Longitude"].fillna(median_longitude)
median_latitude = df["Latitude"].median()
df["Latitude"] = df["Latitude"].fillna(median_latitude)
median_xcordinate = df["X Coordinate"]
df["X Coordinate"] = df["X Coordinate"].fillna(median_xcordinate)
median_ycordinate = df["Y Coordinate"]
df["Y Coordinate"] = df["Y Coordinate"].fillna(median_ycordinate)
df = df.dropna(subset=["X Coordinate"])
df = df.dropna(subset=["Y Coordinate"])
df = df.dropna(subset=["Location"])
df = df.dropna(subset=["Location Description"])
df = df.dropna(subset=["Ward"])
df = df.dropna(subset=["Zip Codes"])
df = df.dropna(subset=["Community Areas"])
df = df.dropna(subset=["Historical Wards 2003-2015"])
df = df.dropna(subset=["Census Tracts"])
df = df.dropna(subset=["Boundaries - ZIP Codes"])
df = df.dropna(subset=["Police Districts"])
df = df.dropna(subset=["Police Beats"])
df = df.dropna(subset=["Arrest"])
df['total_crime'] = df.groupby(['pri_neigh'])['Case Number'].transform('count')
df['total_crime'] = df['total_crime'].astype(int)
# Save the merged CSV file
df.to_csv('Data/Crimes_-_2001_to_Present_modified.csv', index=False)